<a href="https://colab.research.google.com/github/Daniahblog/Colab/blob/master/Copy_of_Pix2PixHD_Next_Frame_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Next Frame Prediction using Pix2PixHD

By Derrick Schultz

Forked repo and tutorial based on [JC Testud’s excellent repo and Medium](https://medium.com/@jctestud/video-generation-with-pix2pix-aed5b1b69f57) article.

## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Install libraries and dependencies



### Run this cell if you haven’t previously install Pix2PixHD in your Google Drive

In [ ]:
%cd /content/drive/My\ Drive
!mkdir nfp-colab
%cd nfp-colab
!git clone -b video https://github.com/dvschultz/pix2pixHD.git
!pip install dominate
%cd pix2pixHD/

/content/drive/My Drive
mkdir: cannot create directory ‘nfp-colab’: File exists
/content/drive/My Drive/nfp-colab
fatal: destination path 'pix2pixHD' already exists and is not an empty directory.
/content/drive/My Drive/nfp-colab/pix2pixHD


### Run this cell if you’ve already installed Pix2PixHD in Google Drive before

In [3]:
%cd /content/drive/My\ Drive/nfp-colab/pix2pixHD/
# !git pull
!pip install dominate


/content/drive/My Drive/nfp-colab/pix2pixHD


## Extract frames from video

Upload a video to either Colab or Google Drive.

* `-video` is the path to the video file
* `-name` should be a unique name (think of it like a project name)
* `-width` and `-height` **must** to be a multiple of 32
* `-p2pdir` leave as `.` unless you know it shouldn’t be that ;)


In [ ]:
!python extract_frames.py -video /content/Short_Video_Zoom.mp4 -name Short_Video_Zoom -p2pdir . -width 1280 -height 736

creating the dataset structure
ffmpeg -v 16 -i /content/Short_Video_Zoom.mp4 -q:v 2 -vf "scale=iw*736/ih:736, crop=1280:736" /content/drive/My\ Drive/nfp-colab/pix2pixHD/datasets/Short_Video_Zoom/train_frames/frame-%06d.jpg -hide_banner
extracting the frames


## Train your model

### Initial training

Note: if you have a large dataset, this may timeout initially (`ValueError: __len__() should return >= 0`). Give it a minute or two and run it again.

*   `--name` should be a unique name (think of it like a project name). For your sanity I recommend using the same `-name` as above.
*   `--dataroot` should point to your dataset. This will usually be in `./datasets/[your project name]`

 



In [ ]:
!python train_video.py --name Short_Video_Zoom --dataroot ./datasets/Short_Video_Zoom --save_epoch_freq 5 #--continue_train

python3: can't open file 'train_video.py': [Errno 2] No such file or directory


### Continue Training
To pick up from a previous training session run the same command you ran to start with and append `--continue_train` to the end of the command.

In [4]:
!python train_video.py --name Short_Video_Zoom --dataroot ./datasets/Short_Video_Zoom --save_epoch_freq 5 --continue_train

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/Short_Video_Zoom
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
fps: 24.0
gpu_ids: [0]
heat_seeking_lvl: 0
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 35
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: Short_Video_Zoom
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: False
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
pstart: 1
pstop: 1
resize_or_crop: scale_width
save_epoch_

#Generating Videos

To generate a video from your completed model, run the `generate_video.py` script. I recommend keeping the `--name` and `--dataroot` arguments the same.

There are a number of additional arguments you’ll need to include in this command:


*   `--fps` frame rate for your video
*   `--how_many` how many frames you want to create (this + fps = video length)
*   `--which_epoch` which epoch you want to generate videos from (note: if you choose `133` but there’s no epoch 133 model file, this will throw an error)
*   `--start_from` by default your video will start predicting images from the 60th frame of your training set. You can pass in the path to a different frame to have it start from that frame

I recommend playing with both the `--which_epoch` and `--start_from` arguments as you can get dramatically different results by changing in the inputs here.





In [5]:
!python generate_video.py --name Short_Video_Zoom --dataroot ./datasets/Short_Video_Zoom  --fps 24 --how_many 600 --which_epoch latest 

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/Short_Video_Zoom
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
fps: 24.0
gpu_ids: [0]
heat_seeking_lvl: 0
how_many: 600
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 35
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: Short_Video_Zoom
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
pstart: 1
pstop: 1
resize_or_crop: scale_width
results_dir: ./results/
scheduled_sampling: False
serial_batches: False
ss_recursion_prob: 0.2
start_from: video
tf_log: False
use_dropout: False
use_enco

In [ ]:
import os

def processFolder(folder, epoch = 10, frameCount = 240, skipCount = 1):
  files = os.listdir(folder)

  count = 0
  for f in files:
    
    if (count % skipCount == 0):
      print(f)
      if epoch == 'latest':
        command = 'python generate_video.py --name glitch_white_circle --dataroot ./datasets/glitch-circle-white_dataset/ --fps 60 --how_many %i --which_epoch latest --start_from %s/%s' % ( frameCount, folder, f)
      else:
        command = 'python generate_video.py --name glitch_white_circle --dataroot ./datasets/glitch-circle-white_dataset/ --fps 24 --how_many %i --which_epoch %i --start_from %s/%s' % ( frameCount, epoch, folder, f)
      os.system(command)
    count += 1

processFolder('./datasets/fuck-white/train_frames',1,600,3050)

frame-018504.jpg
frame-015525.jpg
frame-012746.jpg
frame-009603.jpg
frame-006972.jpg
frame-004082.jpg
frame-001186.jpg
